In [ ]:
import os
cwd = os.getcwd()
nwd = os.path.dirname(cwd)
os.chdir(nwd)

In [ ]:
%load_ext autoreload
%autoreload 2
import numpy as np
from sklearn.linear_model import LinearRegression
from ghostSHAP.ghost_shap import GhostShap

In [ ]:
# Generate data
np.random.seed(seed=5)
p = 10
n = 1000
total_subsets = 2 ** p
total_individuals = total_subsets
ones = np.full(shape=(n, 1), fill_value=1)
X = np.random.normal(loc=0, scale=1, size=(n, p))

X_intercept = np.concatenate((ones, X), axis=1)
beta = np.reshape(np.arange(1, p+2), newshape=(p+1, 1))
#beta = beta[[0, 2, 1], :]
y = np.matmul(X_intercept, beta) + np.random.normal(loc=0, scale=1, size=(n, 1))
beta_hat = np.matmul(np.matmul(np.linalg.inv(np.matmul(X_intercept.T, X_intercept)), X_intercept.T), y)
print('X: \n', X[:3, ])
print('beta_real:\n', beta)
print('beta_hat:\n', beta_hat)

In [ ]:
# Point to be described
x_new = np.full(shape=(1, p), fill_value=2)
x_new_intercept = np.concatenate((np.array([1], ndmin=2), x_new), axis=1)
print(x_new_intercept)

In [ ]:
# Black box model
linear_model = LinearRegression(fit_intercept=True)
linear_model = linear_model.fit(X, y)

In [ ]:
print('real betas:\n', beta)
print('intercept:', linear_model.intercept_)
print('coef:', linear_model.coef_)
print('real vale for x_new:', np.dot(x_new_intercept, beta))
print('predicted value for x_new:', linear_model.predict(x_new))

In [ ]:
ghost_shap = GhostShap(
    predict_fn=linear_model.predict,
    data=X,
    x_test=x_new
)
importance = ghost_shap.get_importance()

In [ ]:
print('individual importance:\n', importance)
print('aggregated importnace:\n', np.sum(importance))
x_new_beta = [x * b for x, b in zip(x_new[0], beta[1:])]
print('x_new * beta:\n', x_new_beta)
print('predicted value for x_new - mean value:\n', linear_model.predict(x_new) - np.mean(linear_model.predict(X)))